# White noise time series

In [1]:
import os
from glob import glob

import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd


# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
# import geopandas as gpd
from shapely.geometry import Polygon

%matplotlib inline
from matplotlib import pyplot as plt


import drifters.utils as ut
import pynsitu as pin
from sstats import signals as sg
from sstats import sigp as sigp
from sstats import tseries as ts
from sstats import get_cmap_colors

import dask.dataframe as dd
from dask.delayed import delayed

In [2]:
from dask.distributed import Client

if True:
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client

    # cluster = PBSCluster()
    # w = cluster.scale(jobs=3) # 2 not enough for lon, lat, year, binning
    cluster = PBSCluster(cores=4, processes=4, walltime="04:00:00")
    w = cluster.scale(jobs=1)
else:
    from dask.distributed import LocalCluster

    cluster = LocalCluster()

client = Client(cluster)
client

/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:237: FutureWarning: extra has been renamed to worker_extra_args. You are still using it (even if only set to []; please also check config files). If you did not set worker_extra_args yet, extra will be respected for now, but it will be removed in a future release. If you already set worker_extra_args, extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.min

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.46:8787/status,
Dashboard: http://10.148.0.46:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.46:56493,Workers: 0
Dashboard: http://10.148.0.46:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [2]:
noise_std = 1
time_unit = pd.Timedelta("1D")
t_ref = pd.Timestamp(2000, 1, 1)
t_size = 5000


t = pd.date_range(
    pd.Timestamp(2000, 1, 1), periods=3700, freq="1H"
)  # mean T_size in GDP trajectories, time in days
draw = 2  # x, y
da = ts.normal(time=t, draws=draw) * noise_std

dax = da.isel(draw=0).rename("nx").drop("draw")
day = da.isel(draw=1).rename("ny").drop("draw")
ds = xr.merge([dax, day])
ds["v_nx"] = ds.nx.differentiate("time") / 86400
ds["v_ny"] = ds.ny.differentiate("time") / 86400
ds["a_nx"] = ds.v_nx.differentiate("time") / 86400
ds["a_ny"] = ds.v_ny.differentiate("time") / 86400
ds["noise_std"] = noise_std
ds["id"] = 0
ds.attrs = {"description": f"white noise with std={noise_std}"}
df = ds.to_dataframe()
df

,nx,ny,v_nx,v_ny,a_nx,a_ny,noise_std,id
time,,,,,,,,
2000-01-01 00:00:00,-1.210650,-0.829293,3.888455e-18,-1.857587e-18,-1.623124e-35,-1.397863e-35,1,0
2000-01-01 01:00:00,-0.324778,0.494298,2.440694e-18,5.816424e-18,-9.922503e-36,-1.378422e-35,1,0
2000-01-01 02:00:00,-0.626219,0.179387,1.292136e-18,1.770794e-18,1.003956e-35,-5.965717e-36,1,0
2000-01-01 03:00:00,0.975323,0.565092,-4.973639e-19,-2.533468e-18,8.454012e-36,5.052195e-37,1,0
2000-01-01 04:00:00,-0.112595,-0.438673,-1.709083e-18,-8.523519e-19,-7.053745e-36,-3.132881e-37,1,0
...,...,...,...,...,...,...,...,...
2000-06-02 23:00:00,-1.214646,0.270685,7.978875e-19,2.584606e-18,-2.304125e-36,1.100358e-36,1,0
2000-06-03 00:00:00,-2.294604,-1.543646,2.942233e-21,-1.003937e-18,-1.452696e-36,3.773576e-36,1,0
2000-06-03 01:00:00,0.124309,1.414132,-2.353534e-18,-2.987350e-18,-9.207321e-37,-3.163304e-36,1,0


# Store in parquet

In [3]:
root_dir = "/home1/datawork/mdemol/GDP"
parquet_noise = os.path.join(root_dir, f"argos_av_noise_{noise_std}.parquet")
df.to_parquet(parquet_noise, engine="pyarrow")

## Visualisation 

In [4]:
df_load = dd.read_parquet(parquet_noise)
df_load.nx.compute()

time
2000-01-01 00:00:00   -1.210650
2000-01-01 01:00:00   -0.324778
2000-01-01 02:00:00   -0.626219
2000-01-01 03:00:00    0.975323
2000-01-01 04:00:00   -0.112595
                         ...   
2000-06-02 23:00:00   -1.214646
2000-06-03 00:00:00   -2.294604
2000-06-03 01:00:00    0.124309
2000-06-03 02:00:00   -0.170907
2000-06-03 03:00:00   -0.011483
Name: nx, Length: 3700, dtype: float64

In [5]:
cluster.close()

NameError: name 'cluster' is not defined